# Proyección Anual del PML

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

ruta = 'C:/Cursos/Data Science/Proyecto CENANCE/pml-cancun-forecasting/data/PML_CANCUN_FINAL.csv'

# Cargar Datos
try:
    df = pd.read_csv(ruta, parse_dates=['fecha'])
    print("✅ Archivo cargado correctamente")
    print(f"📊 Dimensiones: {df.shape}")
    
    # Mostrar las primeras filas 
    print(df.tail())

except Exception as e:
    print(f"❌ Error al cargar el archivo: {str(e)}")
    print("\n🔍 Verifica:")
    print(f"1. Que el archivo existe en: {ruta}")
    print("2. Que el nombre del archivo es exactamente 'PML_CANCUN_FINAL.csv'")
    print("3. Que no hay caracteres especiales en la ruta")
    
def parse_mixed_dates(date_str):
    try:
        # Primero intenta con formato AAAA-MM-DD
        return pd.to_datetime(date_str, format='%Y-%m-%d')
    except ValueError:
        try:
            # Si falla, intenta con formato DD/MM/AAAA
            return pd.to_datetime(date_str, format='%d/%m/%Y')
        except ValueError:
            # Si ambos fallan, devuelve NaT (Not a Time)
            return pd.NaT

# Aplicar la función a la columna de fechas
df['fecha'] = df['fecha'].apply(parse_mixed_dates)

# Verificar si hay fechas no convertidas
if df['fecha'].isna().any():
    print(f"⚠️ Advertencia: {df['fecha'].isna().sum()} fechas no pudieron convertirse")
    print("Registros problemáticos:")
    print(df[df['fecha'].isna()])
else:
    print("✅ Todas las fechas convertidas exitosamente")

# Combinar con la hora (1-24) para crear timestamp completo
df['fecha_hora'] = df['fecha'] + pd.to_timedelta(df['hora'] - 1, unit='h')

# Eliminar filas con fechas inválidas si es necesario
df = df.dropna(subset=['fecha'])

# Verificar resultados
print("\nEjemplo de fechas convertidas:")
print(df[['fecha', 'hora', 'fecha_hora']].head(3))
print("\nEjemplo del registro problemático convertido:")
print(df[df['fecha_hora'].dt.strftime('%d/%m/%Y') == '15/08/2024'].head())

✅ Archivo cargado correctamente
📊 Dimensiones: (17470, 6)
            fecha  hora    zona  precio  energia  congestion
17465  30/09/2020    20  CANCUN  604.48   523.48         0.0
17466  30/09/2020    21  CANCUN  620.15   531.40         0.0
17467  30/09/2020    22  CANCUN  625.37   532.39         0.0
17468  30/09/2020    23  CANCUN  625.96   521.96         0.0
17469  30/09/2020    24  CANCUN  625.87   519.85         0.0
✅ Todas las fechas convertidas exitosamente

Ejemplo de fechas convertidas:
       fecha  hora          fecha_hora
0 2020-04-01     1 2020-04-01 00:00:00
1 2020-04-01     2 2020-04-01 01:00:00
2 2020-04-01     3 2020-04-01 02:00:00

Ejemplo del registro problemático convertido:
          fecha  hora    zona   precio  energia  congestion  \
6093 2024-08-15     1  CANCUN  5770.44  1082.40     4418.25   
6094 2024-08-15     2  CANCUN   970.89   774.01       -0.12   
6095 2024-08-15     3  CANCUN   634.29   508.03       -0.25   
6096 2024-08-15     4  CANCUN   544.40   440.

# --------------------------
# 1. Escenarios y supuestos a largo plazo

Para la proyección anual del Precio Marginal Local (PML), se consideran los siguientes supuestos:

- Se mantiene el comportamiento estacional observado en los datos históricos.
- No se contemplan eventos extraordinarios como cambios regulatorios, desastres naturales o crisis económicas.
- Se espera una demanda energética relativamente estable, sin incrementos abruptos ni decrecimientos marcados.
- Los modelos se entrenan únicamente con series históricas de precios, sin variables exógenas.
- Las proyecciones a 1, 3 y 5 años se hacen con frecuencias horarias y agregaciones anuales posteriores para facilitar el análisis.

Estos supuestos permiten generar escenarios base útiles para análisis exploratorios y planeación energética.

# --------------------------

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings('ignore')

ruta = 'C:/Cursos/Data Science/Proyecto CENANCE/pml-cancun-forecasting/data/PML_CANCUN_FINAL.csv'

# 1. Cargar datos de manera ROBUSTA
try:
    df = pd.read_csv(ruta)
    # Intentamos combinar las columnas 'fecha' y 'hora' directamente al cargar
    df['fecha_hora'] = pd.to_datetime(df['fecha'] + ' ' + df['hora'].astype(str) + ':00', errors='coerce')
    df = df.dropna(subset=['fecha_hora', 'precio']) # Eliminar filas donde no se pudo crear la fecha o no hay precio
    df['precio'] = pd.to_numeric(df['precio'], errors='coerce') # Asegurar que el precio sea numérico
    df = df.dropna(subset=['precio'])
    df = df[['fecha_hora', 'precio']].sort_values(by='fecha_hora').drop_duplicates(subset=['fecha_hora'], keep='first').set_index('fecha_hora')
    df = df.asfreq('H', method='ffill') # Forzar frecuencia horaria llenando huecos con el último valor válido

    print("✅ Datos cargados y preprocesados exitosamente.")
    print(f"📊 Dimensiones después de limpieza: {df.shape}")
    print("\n🔍 Primeras filas:\n", df.head())

except FileNotFoundError:
    print(f"❌ Error: El archivo no se encontró en la ruta: {ruta}")
    exit()
except KeyError as e:
    print(f"❌ Error: No se encontraron las columnas esperadas ({e}). Verifica el nombre de las columnas en tu CSV.")
    exit()
except Exception as e:
    print(f"❌ Error inesperado al cargar y preprocesar los datos: {e}")
    exit()

# 2. Modelado SARIMA con búsqueda automática de parámetros
try:
    print("\n⚙️ Buscando parámetros óptimos para SARIMA...")
    modelo_auto = auto_arima(
    df['precio'],
    seasonal=True,
    m=24,
    stepwise=True,
    suppress_warnings=True,
    trace=True,
    error_action='ignore',
    max_p=2, 
    max_q=2, 
    max_P=1, 
    max_Q=1  
)
    print("\n✅ Parámetros óptimos encontrados:")
    print(modelo_auto.summary())

    # 3. Entrenar el modelo SARIMAX con los parámetros encontrados
    order = modelo_auto.order
    seasonal_order = modelo_auto.seasonal_order

    modelo_sarima = SARIMAX(df['precio'],
                            order=order,
                            seasonal_order=seasonal_order,
                            enforce_stationarity=False,
                            enforce_invertibility=False)
    resultado = modelo_sarima.fit(disp=False)
    print("\n✅ Modelo SARIMAX entrenado.")

    # 4. Generar pronósticos
    horas_pronostico = {
        '1 año': 24 * 365,
        '3 años': 24 * 365 * 3,
        '5 años': 24 * 365 * 5
    }
    predicciones_sarima = {}

    for label, h in horas_pronostico.items():
        pred = resultado.get_forecast(steps=h)
        pred_df = pd.DataFrame({'precio_predicho': pred.predicted_mean})
        pred_df.index = pd.date_range(start=df.index[-1], periods=h, freq='H')
        predicciones_sarima[label] = pred_df.reset_index().rename(columns={'index': 'fecha_hora'})
        print(f"\n✅ Pronóstico para {label} generado.")

    # 5. Visualización
    plt.figure(figsize=(14, 6))
    plt.plot(df[-24 * 30:], label='Histórico (últimos 30 días)')
    for label, df_pred in predicciones_sarima.items():
        plt.plot(df_pred['fecha_hora'], df_pred['precio_predicho'], label=f'Proyección {label}')
    plt.legend()
    plt.title("Proyecciones SARIMA a 1, 3 y 5 años")
    plt.xlabel("Fecha")
    plt.ylabel("Precio")
    plt.tight_layout()
    plt.show()

    # 6. Exportar a CSV
    for label, df_pred in predicciones_sarima.items():
        archivo = f"proyeccion_sarima_{label.replace(' ', '_')}.csv"
        df_pred.to_csv(archivo, index=False)
        print(f"📤 Archivo exportado: {archivo}")

    print("\n🎉 ¡Proceso de proyección anual completado con éxito!")

except Exception as e:
    print(f"\n❌ ERROR CRÍTICO en el modelado o pronóstico: {e}")
    print("Por favor, revisa los pasos anteriores y asegúrate de que los datos estén correctamente cargados y preprocesados.")


✅ Datos cargados y preprocesados exitosamente.
📊 Dimensiones después de limpieza: (51959, 1)

🔍 Primeras filas:
                      precio
fecha_hora                 
2020-01-01 01:00:00  428.23
2020-01-01 02:00:00  396.10
2020-01-01 03:00:00  374.27
2020-01-01 04:00:00  359.29
2020-01-01 05:00:00  327.40

⚙️ Buscando parámetros óptimos para SARIMA...
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[24] intercept   : AIC=798554.260, Time=394.69 sec
 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=799643.194, Time=0.73 sec
 ARIMA(1,1,0)(1,0,0)[24] intercept   : AIC=798954.676, Time=65.87 sec
 ARIMA(0,1,1)(0,0,1)[24] intercept   : AIC=798854.826, Time=88.16 sec
 ARIMA(0,1,0)(0,0,0)[24]             : AIC=799641.194, Time=0.39 sec
 ARIMA(2,1,2)(0,0,1)[24] intercept   : AIC=798552.260, Time=373.51 sec
 ARIMA(2,1,2)(0,0,0)[24] intercept   : AIC=798550.339, Time=7.14 sec

❌ ERROR CRÍTICO en el modelado o pronóstico: Unable to allocate 21.4 MiB for an array with shape (27, 51959) an

# --------------------------
# 2. Modelo Prophet (log)
# --------------------------

In [ ]:
df_log = df.copy()
df_log['y'] = np.log1p(df_log['precio'])
df_log['ds'] = df_log['fecha_hora']
df_prophet_log = df_log[['ds', 'y']]

modelo_log = Prophet()
modelo_log.fit(df_prophet_log)

# ------------------------------
## 3. Proyección a 1, 3 y 5 años
# -------------------------------

In [ ]:
horas_1a = 365 * 24
horas_3a = 3 * horas_1a
horas_5a = 5 * horas_1a

futuro_5a = modelo_log.make_future_dataframe(periods=horas_5a, freq='H')
pronostico_5a = modelo_log.predict(futuro_5a)
pronostico_5a['yhat_real'] = np.expm1(pronostico_5a['yhat'])

# Filtramos para las proyecciones de 1, 3 y 5 años
fecha_inicio = df['fecha_hora'].max()

pronostico_1a = pronostico_5a[pronostico_5a['ds'] <= fecha_inicio + pd.Timedelta(days=365)]
pronostico_3a = pronostico_5a[pronostico_5a['ds'] <= fecha_inicio + pd.Timedelta(days=3*365)]

# --------------------------
# 4. Visualización
# --------------------------

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df['fecha_hora'], df['precio'], label='Precio real', linewidth=1)
plt.plot(pronostico_1a['ds'], pronostico_1a['yhat_real'], label='Proyección 1 año')
plt.plot(pronostico_3a['ds'], pronostico_3a['yhat_real'], label='Proyección 3 años')
plt.plot(pronostico_5a['ds'], pronostico_5a['yhat_real'], label='Proyección 5 años')
plt.legend()
plt.title("Proyección Anual del Precio (PML)")
plt.xlabel("Fecha")
plt.ylabel("Precio")
plt.tight_layout()
plt.show()

# --------------------------
# 6. Métricas de error solo sobre el histórico
# --------------------------

In [ ]:
df_pred = pronostico_5a[['ds', 'yhat_real']].rename(columns={'ds': 'fecha_hora', 'yhat_real': 'precio_predicho'})
df_real = df[['fecha_hora', 'precio']]

comparacion = pd.merge(df_real, df_pred, on='fecha_hora', how='inner')
mae = mean_absolute_error(comparacion['precio'], comparacion['precio_predicho'])
rmse = np.sqrt(mean_squared_error(comparacion['precio'], comparacion['precio_predicho']))
mape = np.mean(np.abs((comparacion['precio'] - comparacion['precio_predicho']) / comparacion['precio'])) * 100

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

# --------------------------
# 7. Exportar para Power BI
# --------------------------

In [ ]:
pronostico_exportar = pronostico_5a[['ds', 'yhat_real']].rename(columns={
    'ds': 'fecha_hora',
    'yalse)hat_real': 'precio_proyectado'
})

# Exportar a CSV
pronostico_exportar.to_csv('proyeccion_anual_pml.csv', index=False